In [1]:
import imaplib, email, os
from email.header import decode_header

imap_url = "imap.gmail.com"
# Environment variables set in ~/.zshrc
usrname = os.environ.get("GM_USR")
passwd = os.environ.get("GM_PWD")
# Where you want your attachments to be saved (ensure this directory exists)
#When there is space in path use "r" string
attachment_dir = r'/Users/pengfeimacbook18/Library/CloudStorage/OneDrive-Personal/Workspace 2/DistWeekly' 

In [2]:
# sets up the auth
def auth(usrname, passwd,imap_url):
    con = imaplib.IMAP4_SSL(imap_url)
    con.login(usrname, passwd)
    return con

In [3]:
con = auth(usrname, passwd,imap_url)

In [ ]:
con.list()

In [4]:

con.select('Weekly')

('OK', [b'19'])

In [5]:
#search for a particular email
def search(key,value,con):
    result, data  = con.search(None,key,'"{}"'.format(value))
    return data

In [7]:

msgb = search('Subject', '*', con)
msgb

[b'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19']

In [8]:
#extracts emails from byte array
def get_emails(result_bytes):
    msgs = []
    for num in result_bytes[0].split():
        typ, data = con.fetch(num, '(RFC822)')
        msgs.append(data)
    return msgs

In [13]:
# allows you to download attachments
def get_attachments(msg):
    for part in msg.walk():
        if part.get_content_maintype()=='multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        fileName = part.get_filename()
        #When filename of attachment is chinese
        if decode_header(fileName)[0][1] == 'utf-8':
            fileName = decode_header(fileName)[0][0].decode("utf-8")
        if decode_header(fileName)[0][1] == 'gb2312':
            fileName = decode_header(fileName)[0][0].decode("gb2312")
        # print(fileName)
        # print(type(fileName))
        # print(decode_header(fileName))
        # print(type(decode_header(fileName)))
        # print(decode_header(fileName)[0][0].decode("utf-8"))
        # print(type(decode_header(fileName)[0][0]))
        
        if bool(fileName):
            filePath = os.path.join(attachment_dir, fileName)
            with open(filePath,'wb') as f:
                f.write(part.get_payload(decode=True))


In [14]:
msgs = get_emails(msgb)
for msg in msgs:
    raw = email.message_from_bytes(msg[0][1])
    get_attachments(raw)


[('上海博通代理周报-20220316.doc', None)]
[('上海博通代理周报-20220307.doc', None)]
[('上海博通代理周报0309（1）.docx', None)]
[('2022.3.9上海博通代理周报.doc', None)]
[('上海博通代理周报3月第一周.doc', None)]
[('上海博通代理周报-20220301.doc', None)]
[('副本-上海博通代理周报0302.docx', None)]
[('上海博通代理周报0221-0225.doc', None)]
[('上海博通代理周报0214-0218.doc', None)]
[('上海博通代理周报0223.docx', None)]
[('上海博通代理周报-芯伍科技.doc', None)]
[('上海博通代理周报0216(1).docx', None)]
[('上海博通代理周报.doc', None)]


In [ ]:
# extracts the body from the email
def get_body(msg):
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else:
        return msg.get_payload(None,True)


In [ ]:
get_body(raw)

In [ ]:
con.select('test')
msgb = search('From', 'sam@bekencorp.com', con)
msgs = get_emails(msgb)
for msg in msgs:
    raw = email.message_from_bytes(msg[0][1])
    get_attachments(raw)